Recogida de datos

In [139]:
import sqlite3

def guardar(tabla, url, informacion):
    with sqlite3.connect('datosScraping.db') as conexion:
        cursor = conexion.cursor()
    
        # Verificar si ya existe el registro antes de la inserción
        cursor.execute(f"SELECT COUNT(*) FROM {tabla} WHERE url = ? AND informacion = ?", (url, informacion))
        registroExistente = cursor.fetchone()[0]

        if not registroExistente:
            # Evitar posibles problemas de SQL injection utilizando parámetros de la consulta
            cursor.execute(f"INSERT INTO {tabla} (url, informacion) VALUES (?, ?)", (url, informacion))
            print(f"Guardado en {tabla}: {url}, {informacion}")
            conexion.commit()
        else:
            print(f"Registro existente en {tabla}: {url}, {informacion}")

            
    conexion.close()

In [140]:
#CREAMOS UN ARCHIVO PARA GUARDAR LA URL:
def listaUrls(urlNueva):
    # Verificamos si la URL ya existe en el archivo

    try:
        # Verificamos si el archivo existe
        archivo = open("urls.txt", 'r')
        urlsExistentes = archivo.read().splitlines()
        archivo.close()
    except FileNotFoundError:
        # Si el archivo no existe, lo creamos
        archivo = open("urls.txt", 'w')
        urlsExistentes = []
        archivo.close()


    if urlNueva not in urlsExistentes:
        # Si la URL no existe, añadir al final del archivo
           
        archivo = open("urls.txt", 'a')
        archivo.write(f"{urlNueva}\n")
        print(f"URL añadida: {urlNueva}")
        archivo.close()
        return True
            
    else:
        print(f"La URL ya existe en el archivo: {urlNueva}")
        return False



In [209]:
def raspado(urls, etiquetas):
    import requests
    from bs4 import BeautifulSoup
    from urllib.parse import urlparse

    for url in urls: #Verificamos si hay alguna url inválida
        parsed_url = urlparse(url)
        if not (parsed_url.scheme and parsed_url.netloc):
            print(f"URL inválida: {url}")
            continue
        if listaUrls(url):
            contenido = requests.get(url)
            sopa = BeautifulSoup(contenido.text, 'html.parser')

            for etiqueta in etiquetas:
                elementos = sopa.find_all(etiqueta)
                
                for elemento in elementos:
                    if etiqueta == 'a':
                        # Si la etiqueta es 'a' (enlace), obtener la URL del atributo 'href'
                        informacion = elemento.get('href')
                        if informacion is not None and not informacion.startswith(('https:', '//',"http:")):
                            continue
                    elif etiqueta == 'img':
                        # Si la etiqueta es 'img' (imagen), obtener la URL del atributo 'src'
                        informacion = elemento.get('src')
                        # Agregar condición para verificar si la URL comienza con 'https' o '//'
                        if informacion is not None and not informacion.startswith(('https:', '//',"http:")):
                            continue  # Salta a la siguiente iteración si no cumple con la condición
                    else:
                        # En otros casos, obtener el texto del elemento
                        informacion = elemento.text.strip()

                    # Guardar en la base de datos
                    guardar(etiqueta, url, informacion)


In [257]:
listaUrl =  ["https://es.wikipedia.org/wiki/D%C3%B3berman","https://www.zooplus.es/magazine/perros/razas-de-perro/doberman","https://www.purina.es/encuentra-mascota/razas-de-perro/dobernmann",
             "https://es.wikipedia.org/wiki/Dogo_argentino","https://www.zooplus.es/magazine/perros/razas-de-perro/dogo-argentino","https://purina.cl/purina/dogo-argentino",
             "https://es.wikipedia.org/wiki/Fox_terrier","https://www.purina.es/encuentra-mascota/razas-de-perro/fox-terrier-de-pelo-duro","https://www.tiendanimal.es/articulos/fox-terrier/",
             "https://es.wikipedia.org/wiki/Galgo_espa%C3%B1ol","https://www.purina.es/encuentra-mascota/razas-de-perro/galgo","https://es.wikipedia.org/wiki/Galgo_ingl%C3%A9s",
             "https://es.wikipedia.org/wiki/Golden_retriever","https://www.purina.es/encuentra-mascota/razas-de-perro/golden-retriever","https://www.zooplus.es/magazine/perros/razas-de-perro/golden-retriever"

             ]
listaEtiquetas = ["h1","p","a","img",]

In [258]:
raspado(listaUrl, listaEtiquetas)

URL añadida: https://es.wikipedia.org/wiki/D%C3%B3berman
Guardado en h1: https://es.wikipedia.org/wiki/D%C3%B3berman, Dóberman
Guardado en p: https://es.wikipedia.org/wiki/D%C3%B3berman, El dóberman (en alemán: dobermann)[7]​ es una raza de perro de origen alemán que debe su nombre a Karl Friedrich Louis Dobermann, un alemán que, a finales del siglo XIX, toma la iniciativa de crear una nueva raza de perro que sirviera eficazmente a su difícil trabajo como recaudador de impuestos. Para conseguir el nacimiento de esta nueva raza, mediante cruces de otras razas caninas como el Rottweiler, el Beauceron, el Weimaraner, el Manchester terrier, o el Pinscher, entre otros. La principal función de esta raza en sus orígenes fue la de proteger a su criador frente a potenciales asaltantes interesados en la recaudación de los impuestos que solía portar. Actualmente sus principales funciones son: perro policía, perro militar, perro de defensa y seguridad, perro guardián, entre otras funciones, según 

Creación de contenido

In [213]:
#Vamos a realizar una funcion que escoja los párrafos en los que mas veces se repita la palabra indicada
import re

def obtenerTresConMasCoincidencias(lista, palabra):
    # Definir una expresión regular para buscar la palabra clave ignorando mayúsculas y minusculas
    patron = re.compile(re.escape(palabra), flags=re.IGNORECASE)

    # Creamos otra función para contar las veces que aparece las palabra clave en una cadena (ignorando mayúsculas y minusculas)
    def veces(cadena):
        return len(re.findall(patron, cadena.lower()))

    #Ordenamos los str por la cantidad de veces que se repite palabra
    listaOrdenada = sorted(lista, key=lambda x: veces(x), reverse=True)

    # Tomar las tres cadenas con más ocurrencias
    top5 = listaOrdenada[:5]

    return top5

In [264]:
import sqlite3

def generarContenido(palabra):
    # Establecer conexión con la base de datos
    with sqlite3.connect('datosScraping.db') as conexion:
        cursor = conexion.cursor()

        # Buscar información en la base de datos utilizando la cláusula LIKE
        #Buscar un título
        cursor.execute("SELECT * FROM h1 WHERE informacion LIKE ?", ('%' + palabra + '%',))
        h1 = cursor.fetchall()

        #Buscar un parrafos
        cursor.execute("SELECT * FROM p WHERE informacion LIKE ?", ('%' + palabra + '%',))
        p = cursor.fetchall()
        
        #Busca imágenes, vamos a quitar los espacios para que salgan las palabras
        palabraSinEspacios = palabra.replace(' ', '')
        cursor.execute("SELECT * FROM img WHERE informacion LIKE ?", ('%' + palabraSinEspacios + '%',))
        img = cursor.fetchall()

        #Escoger el mejor contenido
        parrafos = []
        for row in p:
            parrafo = row[2]
            parrafos.append(parrafo)

        #Hacemos otro con las img:
        imagenes = []
        for row in img:
            imagen = row[2]

            if not imagen.startswith(("http:", "https:")):
            # Si no comienza con "http:" lo agregamos
                imagen = "https:" + imagen

            imagenes.append(imagen)
        
        print(imagenes)

    top3Parrafo = obtenerTresConMasCoincidencias(parrafos,palabra)
    top3Imagen = obtenerTresConMasCoincidencias(imagenes,palabra)

    # Generar un archivo HTML con el contenido obtenido
    with open('plantilla.html', 'r', encoding='utf-8') as template_file:
        template = template_file.read()
        
        #titulo
        template = template.replace('<!--Título-->', f'{h1[0][2]}' if h1 else palabra)
        #enlace
        template = template.replace('Enlace a la web', f'{h1[0][1]}' if h1 else '')
        #parrafo
        template = template.replace('<!--Texto_Principal-->', f'{top3Parrafo[0]}' if top3Parrafo else '')
        #Enlace al párrafo ###########Pendiente de mejorar
        template = template.replace('+info', f'{p[0][1]}' if top3Parrafo else '')
        print(p[0][1])
        #Imagen de logo
        template = template.replace('Imagen logo', f'{top3Imagen[0]}' if top3Imagen else '')
        #imagen principal
        template = template.replace('Imagen principal', f'{top3Imagen[1]}' if top3Imagen and len(top3Imagen) > 1 else '')
        #Texto carrusel
        template = template.replace('<!--Texto_carrusel-->', f'{top3Parrafo[1]}' if top3Parrafo else '')
        #Imagenes carrusel
        template = template.replace('Imagen1', f'{top3Imagen[2]}' if top3Imagen and len(top3Imagen) > 2 else '')
        template = template.replace('Imagen2', f'{top3Imagen[3]}' if top3Imagen and len(top3Imagen) > 3 else '')
        template = template.replace('Imagen3', f'{top3Imagen[4]}' if top3Imagen and len(top3Imagen) > 4 else '')
        #Pendiente de lista de enlaces relacionados
    with open('output.html', 'w', encoding='utf-8') as output_file:
        output_file.write(template)




In [272]:
generarContenido('galgo')

['https://upload.wikimedia.org/wikipedia/commons/thumb/4/41/Galgo_espagnol_005.jpg/320px-Galgo_espagnol_005.jpg', 'https://upload.wikimedia.org/wikipedia/commons/thumb/e/e1/Galgo_Espa%C3%B1ol_perfil_001.jpg/285px-Galgo_Espa%C3%B1ol_perfil_001.jpg', 'https://upload.wikimedia.org/wikipedia/commons/thumb/6/65/Fresco_soriano_con_galgos_de_la_ermita_moz%C3%A1rabe_de_San_Baudilio.jpg/220px-Fresco_soriano_con_galgos_de_la_ermita_moz%C3%A1rabe_de_San_Baudilio.jpg', 'https://upload.wikimedia.org/wikipedia/commons/thumb/0/09/Galgo_Espa%C3%B1ol_en_los_campos_castellanos.jpg/220px-Galgo_Espa%C3%B1ol_en_los_campos_castellanos.jpg', 'https://upload.wikimedia.org/wikipedia/commons/thumb/6/69/Galgo_espa%C3%B1ol_cachorro.jpg/250px-Galgo_espa%C3%B1ol_cachorro.jpg', 'https://upload.wikimedia.org/wikipedia/commons/thumb/4/40/Galgo_Espa%C3%B1ol_fotograf%C3%ADa_blanco_y_negro_ni%C3%B1os.jpg/220px-Galgo_Espa%C3%B1ol_fotograf%C3%ADa_blanco_y_negro_ni%C3%B1os.jpg', 'https://upload.wikimedia.org/wikipedia/commo